In [8]:
from typing import (
    Union,
    Optional,
    Tuple,
)

import numpy as np
import torch
from torch import nn
from torch.nn.common_types import Tensor

In [27]:
inputs = torch.randn(1, 1, 5, 5)
filters = torch.randn(1, 1, 3, 3)

In [ ]:
nn.functional.conv2d(inputs, filters, padding=1)

In [35]:
def convolution2D(
    tensor: Tensor,
    weights: Tensor,
    bias: Optional[bool] = None,
    stride: Optional[Union[int, Tuple]] = 1,
    padding: Optional[Union[int, Union[Tuple, str]]] = 0,
    dilation: Optional[Union[int, Tuple]] = 1,
    groups: Optional[int] = 1,
) -> Tensor:
    tensor_2d = tensor[0, 0].copy()
    weights_2d = weights[0, 0].copy()

    if bias:
        bias_value = torch.rand(weights_2d)
    else:
        bias_value = torch.zero(weights_2d)

    if padding > 0:
        tensor_2d = nn.functional.pad(tensor_2d, padding)

    
    input_height, input_width = tensor_2d.shape
    weights_height, weights_width = weights_2d.shape

    Y = torch.zeros(
        (input_height + weights_height - 1, input_width + weights_width - 1)
    )

    for i in range(input_height):
        for j in range(input_width):
            Y[i : i + weights_height, j : j + weights_width] += (
                weights_2d[i, j] * weights_2d
            )

    return Y[
        weights_height - 1 : -weights_height + 1, weights_width - 1 : -weights_width + 1
    ]

In [36]:
convolution2D(inputs[0,0].numpy(), filters[0,0].numpy())

IndexError: index 3 is out of bounds for axis 1 with size 3